# `l2hmc-qcd`

This notebook contains a minimal working example for the 4D $SU(3)$ model

Uses `torch.complex128` by default

## Setup

In [1]:
# %matplotlib inline
# import matplotlib_inline
# matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import lovely_tensors as lt
lt.monkey_patch()
lt.set_config(color=False)
# automatically detect and reload local changes to modules
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from l2hmc.utils.plot_helpers import FigAxes, set_plot_style
set_plot_style()

Using device: cpu


In [2]:
import os
from pathlib import Path
from typing import Optional
from rich import print

import lovely_tensors as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml

from l2hmc.utils.dist import setup_torch
seed = np.random.randint(2 ** 32)
print(f"seed: {seed}")
_ = setup_torch(precision='float64', backend='DDP', seed=seed)

import l2hmc.group.su3.pytorch.group as g
from l2hmc.utils.rich import get_console
from l2hmc.common import grab_tensor, print_dict
from l2hmc.configs import dict_to_list_of_overrides, get_experiment
from l2hmc.experiment.pytorch.experiment import Experiment, evaluate  # noqa  # noqa
from l2hmc.utils.plot_helpers import set_plot_style

os.environ['COLORTERM'] = 'truecolor'
os.environ['MASTER_PORT'] = '5439'
# os.environ['MPLBACKEND'] = 'module://matplotlib-backend-kitty'
# plt.switch_backend('module://matplotlib-backend-kitty')
console = get_console()


set_plot_style()

from l2hmc.utils.plot_helpers import (  # noqa
    set_plot_style,
    plot_scalar,
    plot_chains,
    plot_leapfrogs
)

def savefig(fig: plt.Figure, fname: str, outdir: os.PathLike):
    pngfile = Path(outdir).joinpath(f"pngs/{fname}.png")
    svgfile = Path(outdir).joinpath(f"svgs/{fname}.svg")
    pngfile.parent.mkdir(exist_ok=True, parents=True)
    svgfile.parent.mkdir(exist_ok=True, parents=True)
    fig.savefig(svgfile, transparent=True, bbox_inches='tight')
    fig.savefig(pngfile, transparent=True, bbox_inches='tight', dpi=300)

def plot_metrics(metrics: dict, title: Optional[str] = None, **kwargs):
    outdir = Path(f"./plots-4dSU3/{title}")
    outdir.mkdir(exist_ok=True, parents=True)
    for key, val in metrics.items():
        fig, ax = plot_metric(val, name=key, **kwargs)
        if title is not None:
            ax.set_title(title)
        console.log(f"Saving {key} to {outdir}")
        savefig(fig, f"{key}", outdir=outdir)
        plt.show()

def plot_metric(
        metric: torch.Tensor,
        name: Optional[str] = None,
        **kwargs,
):
    assert len(metric) > 0
    if isinstance(metric[0], (int, float, bool, np.floating)):
        y = np.stack(metric)
        return plot_scalar(y, ylabel=name, **kwargs)
    element_shape = metric[0].shape
    if len(element_shape) == 2:
        y = grab_tensor(torch.stack(metric))
        return plot_leapfrogs(y, ylabel=name)
    if len(element_shape) == 1:
        y = grab_tensor(torch.stack(metric))
        return plot_chains(y, ylabel=name, **kwargs)
    if len(element_shape) == 0:
        y = grab_tensor(torch.stack(metric))
        return plot_scalar(y, ylabel=name, **kwargs)
    raise ValueError

seed: 1444224244
[07/20/23 09:22:51][WARNING][dist.py:332] - Setting default dtype: float64
[07/20/23 09:22:51][INFO][dist.py:338] - Global Rank: 0 / 0


## Load config + build Experiment

In [3]:
from rich import print
set_plot_style()

from l2hmc.configs import CONF_DIR
su3conf = Path(f"{CONF_DIR}/su3test.yaml")
with su3conf.open('r') as stream:
    conf = dict(yaml.safe_load(stream))
# overrides = {
#     'backend': 'DDP',
#     'dynamics': {
#         'eps': 0.15,
#         'merge_directions': True,
#     },
#     'network': {
#         'use_batch_norm': True,
#     },
#     'loss': {
#         'use_mixed_loss': False,
#     },
#     'net_weights': {
#         'x': {
#             's': 0.0,
#             't': 0.0,
#             'q': 0.0,
#         },
#         'v': {
#             's': 1.0,
#             't': 1.0,
#             'q': 1.0,
#         },
#     }
# }
# conf |= overrides
console.print(conf)

{
    'annealing_schedule': {'beta_final': 6.0, 'beta_init': 6.0},
    'backend': 'DDP',
    'conv': 'none',
    'dynamics': {
        'eps': 0.01,
        'eps_fixed': False,
        'group': 'SU3',
        'latvolume': [4, 4, 4, 8],
        'nchains': 4,
        'nleapfrog': 4,
        'use_separate_networks': True,
        'use_split_xnets': True,
        'verbose': True
    },
    'framework': 'pytorch',
    'init_aim': False,
    'init_wandb': False,
    'learning_rate': {'clip_norm': 10.0, 'lr_init': 0.001},
    'loss': {'charge_weight': 0.0, 'plaq_weight': 0.0, 'rmse_weight': 1.0, 'use_mixed_loss': False},
    'net_weights': {'v': {'q': 1.0, 's': 1.0, 't': 1.0}, 'x': {'q': 0.0, 's': 0.0, 't': 0.0}},
    'network': {'activation_fn': 'tanh', 'dropout_prob': 0.0, 'units': [64, 64], 'use_batch_norm': False},
    'restore': False,
    'save': False,
    'steps': {'log': 1, 'nepoch': 10, 'nera': 1, 'print': 1, 'test': 50},
    'use_tb': False,
    'use_wandb': False
}



In [4]:
overrides = dict_to_list_of_overrides(conf)
ptExpSU3 = get_experiment(overrides=[*overrides], build_networks=True)
console.print(ptExpSU3.config)
state = ptExpSU3.trainer.dynamics.random_state(6.0)
console.print(f"checkSU(state.x): {g.checkSU(state.x)}")
console.print(f"checkSU(state.x): {g.checkSU(g.projectSU(state.x))}")
assert isinstance(state.x, torch.Tensor)
assert isinstance(state.beta, torch.Tensor)
assert isinstance(ptExpSU3, Experiment)

[07/20/23 09:23:05][INFO][dist.py:226] - Caught MASTER_PORT:5439 from environment!
[07/20/23 09:23:05][INFO][dist.py:226] - Caught MASTER_PORT:5439 from environment!
[07/20/23 09:23:05][WARNING][trainer.py:437] - Using `torch.optim.Adam` optimizer
[07/20/23 09:23:05][INFO][trainer.py:285] - num_params in model: 89762056
[07/20/23 09:23:06][WARNING][trainer.py:251] - logging with freq 1 for wandb.watch
ExperimentConfig(
    {'setup': {'id': None, 'group': None, 'config': None, 'save_code': True, 'sync_tensorboard': True, 'mode': 'online', 'resume': 'allow', 'entity': 'l2hmc-qcd', 'project': 'l2hmc-qcd', 'settings': {'start_method': 'thread'}, 'tags': ['beta_init=6.0', 'beta_final=6.0']}},
    Steps(1, 10, 50, log=1, print=1, extend_last_era=1),
    'pytorch',
    LossConfig(charge_weight=0.0, rmse_weight=1.0),
    NetworkConfig([64, 64], 'tanh', 0.0, use_batch_norm=False),
    ConvolutionConfig(filters=[], sizes=[], pool=[]),
    NetWeights(x=NetWeight(s=0.0, t=0.0, q=0.0)),
    Dynamic

## HMC

In [ ]:
from l2hmc.utils.plot_helpers import set_plot_style
set_plot_style()

In [ ]:
from l2hmc.common import get_timestamp
TSTAMP = get_timestamp()
OUTPUT_DIR = Path(f"./outputs/pt4dSU3/{TSTAMP}")
HMC_DIR = OUTPUT_DIR.joinpath('hmc')
EVAL_DIR = OUTPUT_DIR.joinpath('eval')
TRAIN_DIR = OUTPUT_DIR.joinpath('train')
HMC_DIR.mkdir(exist_ok=True, parents=True)
EVAL_DIR.mkdir(exist_ok=True, parents=True)
TRAIN_DIR.mkdir(exist_ok=True, parents=True)

In [5]:
xhmc, history_hmc = evaluate(
    nsteps=100,
    exp=ptExpSU3,
    beta=state.beta,
    x=state.x,
    eps=0.1,
    nleapfrog=4,
    job_type='hmc',
    nlog=5,
    nprint=10,
    grab=True
)

[07/20/23 09:28:52][INFO][experiment.py:117] - Running 100 steps of hmc at beta=6.0000
[07/20/23 09:28:52][INFO][experiment.py:117] - Running 100 steps of hmc at beta=6.0000
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 0
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 0
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 1
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 1
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 2
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 2
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 3
[07/20/23 09:28:52][INFO][experiment.py:121] - STEP: 3
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 4
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 4
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 5
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 5
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 6
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 6
[07/20/23 09:28:53][INFO][experiment.py:121] - STEP: 7
[

In [6]:
import l2hmc.utils.plot_helpers as ph
xhmc = ptExpSU3.trainer.dynamics.unflatten(xhmc)
console.log(f"checkSU(x_hmc): {g.checkSU(xhmc)}")
dataset_hmc = history_hmc.get_dataset()
ph.plot_dataset(dataset_hmc, outdir=HMC_DIR)
# plot_metrics(history_hmc, title='HMC', marker='.')

[09:34:20] checkSU(x_hmc): (tensor[4] f64 x∈[1.535e-16, 3.394e-16] μ=2.424e-16 σ=7.754e-17 [1.535e-16, 2.570e-16,  
           2.197e-16, 3.394e-16], tensor[4] f64 x∈[7.452e-16, 9.438e-16] μ=8.309e-16 σ=8.260e-17 [7.452e-16,       
           8.165e-16, 8.182e-16, 9.438e-16])                                                                       

[07/20/23 09:34:21][INFO][plot_helpers.py:1018] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 09:34:21][INFO][plot_helpers.py:1018] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 09:34:21][INFO][plot_helpers.py:1018] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/logprob_ridgeplot.svg
[07/20/23 09:34:21][INFO][plot_helpers.py:1018] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/logprob_ridgeplot.svg
[07/20/23 09:34:22][INFO][plot_helpers.py:1018] - Saving figure to: outputs/p

<xarray.Dataset>
Dimensions:    (chain: 4, leapfrog: 5, draw: 19)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * leapfrog   (leapfrog) int64 0 1 2 3 4
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
Data variables:
    energy     (chain, leapfrog, draw) float64 -7.282e+03 ... -1.033e+04
    logprob    (chain, leapfrog, draw) float64 -7.282e+03 ... -1.033e+04
    logdet     (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    acc        (chain, draw) float64 1.0 1.0 1.0 ... 1.0 0.0003556 0.006828
    sumlogdet  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    acc_mask   (chain, draw) float32 1.0 1.0 1.0 0.0 0.0 ... 0.0 1.0 1.0 0.0 0.0
    plaqs      (chain, draw) float64 0.3914 0.4412 0.4666 ... 0.5545 0.5545
    sinQ       (chain, draw) float64 -0.002635 -3.447e-05 ... 0.002555 0.002555
    intQ       (chain, draw) float64 -0.07688 -0.001006 ... 0.07456 0.07456
    dQint      (chain, draw) float64 0.02149 0.03452 0.0

In [7]:
history_hmc.plot_all(title="HMC", outdir=HMC_DIR)

[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:25:53][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/hmc/ridgeplots/svgs/energy_ridgeplot.svg

<xarray.Dataset>
Dimensions:    (chain: 4, leapfrog: 5, draw: 19)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * leapfrog   (leapfrog) int64 0 1 2 3 4
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
Data variables:
    energy     (chain, leapfrog, draw) float64 -7.282e+03 ... -1.033e+04
    logprob    (chain, leapfrog, draw) float64 -7.282e+03 ... -1.033e+04
    logdet     (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    acc        (chain, draw) float64 1.0 1.0 1.0 ... 1.0 0.0003556 0.006828
    sumlogdet  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    acc_mask   (chain, draw) float32 1.0 1.0 1.0 0.0 0.0 ... 0.0 1.0 1.0 0.0 0.0
    plaqs      (chain, draw) float64 0.3914 0.4412 0.4666 ... 0.5545 0.5545
    sinQ       (chain, draw) float64 -0.002635 -3.447e-05 ... 0.002555 0.002555
    intQ       (chain, draw) float64 -0.07688 -0.001006 ... 0.07456 0.07456
    dQint      (chain, draw) float64 0.02149 0.03452 0.0

In [8]:
# history_hmc.plot_dataArray1(dataset_hmc.plaqs, key='Plaqs (HMC)')
# ph.plot_array(dataset_hmc.plaqs.values, key='Plaqs (HMC)')

(<Figure size 1240x480 with 1 Axes>, <Axes: >)


## Evaluation

In [9]:
state = ptExpSU3.trainer.dynamics.random_state(6.0)
ptExpSU3.trainer.dynamics.init_weights(
    # method='zeros',
    # constant=0.0,
    # method='uniform',
    # min=-1e-3,
    # max=1e-3,
    # bias=True,
    xeps=0.05,
    veps=0.05,
)
xeval, history_eval = evaluate(
    nsteps=100,
    exp=ptExpSU3,
    beta=6.0,
    x=state.x,
    job_type='eval',
    nlog=5,
    nprint=10,
    grab=True,
)

[07/20/23 10:47:00][INFO][experiment.py:117] - Running 100 steps of eval at beta=6.0000
[07/20/23 10:47:00][INFO][experiment.py:117] - Running 100 steps of eval at beta=6.0000
[07/20/23 10:47:00][INFO][experiment.py:121] - STEP: 0
[07/20/23 10:47:00][INFO][experiment.py:121] - STEP: 0
[07/20/23 10:47:00][INFO][experiment.py:121] - STEP: 1
[07/20/23 10:47:00][INFO][experiment.py:121] - STEP: 1
[07/20/23 10:47:01][INFO][experiment.py:121] - STEP: 2
[07/20/23 10:47:01][INFO][experiment.py:121] - STEP: 2
[07/20/23 10:47:02][INFO][experiment.py:121] - STEP: 3
[07/20/23 10:47:02][INFO][experiment.py:121] - STEP: 3
[07/20/23 10:47:03][INFO][experiment.py:121] - STEP: 4
[07/20/23 10:47:03][INFO][experiment.py:121] - STEP: 4
[07/20/23 10:47:03][INFO][experiment.py:121] - STEP: 5
[07/20/23 10:47:03][INFO][experiment.py:121] - STEP: 5
[07/20/23 10:47:04][INFO][experiment.py:121] - STEP: 6
[07/20/23 10:47:04][INFO][experiment.py:121] - STEP: 6
[07/20/23 10:47:05][INFO][experiment.py:121] - STEP: 7

In [ ]:
dataset_eval = history_eval.get_dataset()

In [10]:
# plot_metrics(history_eval, title='Evaluate', marker='.')
history_eval.plot_all(outdir=EVAL_DIR, title='Eval')

xeval = ptExpSU3.trainer.dynamics.unflatten(xeval)
console.log(f"checkSU(x_eval): {g.checkSU(xeval)}")
console.log(f"checkSU(x_eval): {g.checkSU(g.projectSU(xeval))}")

[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 10:58:26][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/eval/ridgeplots/svgs/energy_ridgep

In [11]:
# plt.rcParams['figure.dpi'] = 300
# plot_metric(np.stack(history_eval.history['plaqs']), name='Plaqs (Eval)', marker='.')

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[74], line 2
      1 plt.rcParams['figure.dpi'] = 300
----> 2 plot_metric(np.stack(history_eval.history['plaqs']), name='Plaqs (Eval)', marker='.')

Cell In[4], line 73, in plot_metric(metric, name, **kwargs)
     71     return plot_leapfrogs(y, ylabel=name)
     72 if len(element_shape) == 1:
---> 73     y = grab_tensor(torch.stack(metric))
     74     return plot_chains(y, ylabel=name, **kwargs)
     75 if len(element_shape) == 0:

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not numpy.ndarray


## Training

In [12]:
ptExpSU3.trainer.dynamics.init_weights(
    # method='xavier_uniform',
    # constant=0.0,
    # method='uniform',
    # min=-1e-6,
    # max=1e-6,
    # bias=True,
    xeps=0.05,
    veps=0.05,
)
# ptExpSU3.trainer.optimizer.zero_grad()
ptExpSU3.trainer.print_grads_and_weights()

[07/20/23 11:16:26][INFO][trainer.py:2121] - --------------------------------------------------------------------------------
[07/20/23 11:16:26][INFO][trainer.py:2122] - GRADS:
[07/20/23 11:16:26][INFO][common.py:97] - networks.xnet.0.first.input_layer.xlayer.weight: None None 
None
networks.xnet.0.first.input_layer.xlayer.bias: None None 
None
networks.xnet.0.first.input_layer.vlayer.weight: None None 
None
networks.xnet.0.first.input_layer.vlayer.bias: None None 
None
networks.xnet.0.first.hidden_layers.0.weight: None None 
None
networks.xnet.0.first.hidden_layers.0.bias: None None 
None
networks.xnet.0.first.scale.coeff: None None 
None
networks.xnet.0.first.scale.layer.weight: None None 
None
networks.xnet.0.first.scale.layer.bias: None None 
None
networks.xnet.0.first.transf.coeff: None None 
None
networks.xnet.0.first.transf.layer.weight: None None 
None
networks.xnet.0.first.transf.layer.bias: None None 
None
networks.xnet.0.first.transl.weight: None None 
None
networks.xnet.0.

In [13]:
from l2hmc.utils.history import BaseHistory
# history = {}
history: BaseHistory = BaseHistory()
# state = ptExpSU3.trainer.dynamics.random_state(6.0)
# x = state.x
freq = {'print': 5, 'save': 5}
for step in range(100):
    console.print(f'TRAIN STEP: {step}')
    x, metrics = ptExpSU3.trainer.train_step((x, state.beta))
    if (step > 0 and step % freq['print'] == 0):
        print_dict(metrics, grab=True)
    if (step > 0 and step % freq['save'] == 0):
        history.update(metrics)
        # for key, val in metrics.items():
        #     try:
        #         history[key].append(val)
        #     except KeyError:
        #         history[key] = [val]
# plot_metrics(history, title='train', marker='.')

TRAIN STEP: 0

TRAIN STEP: 1

TRAIN STEP: 2

TRAIN STEP: 3

TRAIN STEP: 4

TRAIN STEP: 5

[07/20/23 11:14:17][INFO][common.py:97] - energy: torch.Size([9, 4]) torch.float64 
[[-9171.37842821 -5353.76897792 -9731.61150597 -8698.55691276]
 [-9045.99306112 -5384.02122852 -9571.10269974 -8574.71252864]
 [-8743.96534469 -5462.76512169 -9185.85136864 -8286.05372387]
 [-8365.05862607 -5553.73684832 -8703.60964007 -7923.66304299]
 [-8012.8089365  -5611.89434942 -8255.69415753 -7582.40682846]
 [-7760.91157321 -5612.1309424  -7935.72296509 -7328.48556722]
 [-7625.98774797 -5543.95860552 -7763.41650614 -7185.13144385]
 [-7597.27459964 -5433.19683735 -7728.73435343 -7147.89657596]
 [-7646.83340288 -5318.74717106 -7801.75397426 -7198.76452801]]
logprob: torch.Size([9, 4]) torch.float64 
[[-9171.37842821 -5353.76897792 -9731.61150597 -8698.55691276]
 [-9052.935355   -5390.78579514 -9578.18199679 -8581.39985622]
 [-8758.12886666 -5477.01141859 -9200.61212545 -8300.36964796]
 [-8386.07106637 -5575.667

In [14]:
dataset = history.get_dataset()

[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:13:25][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy

In [15]:
history.plot_all(outdir=TRAIN_DIR, title='Train')

xeval = ptExpSU3.trainer.dynamics.unflatten(xeval)
console.log(f"checkSU(x_train): {g.checkSU(x)}")
console.log(f"checkSU(x_train): {g.checkSU(g.projectSU(x))}")

[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy_ridgeplot.svg
[07/20/23 11:09:31][INFO][plot_helpers.py:1028] - Saving figure to: outputs/pt4dSU3/2023-07-20-092918/train/ridgeplots/svgs/energy

In [16]:
x = ptExpSU3.trainer.dynamics.unflatten(x)
console.print(f"checkSU(x_train): {g.checkSU(x)}")
dataset = history.get_dataset()

checkSU(x_train): (tensor[4] f64 x∈[1.439e-16, 1.487e-16] μ=1.461e-16 σ=2.449e-18 [1.441e-16, 1.476e-16, 1.439e-16, 1.487e-16], tensor[4] f64 x∈[7.127e-16, 8.391e-16] μ=7.605e-16 σ=5.458e-17 [7.451e-16, 8.391e-16, 7.127e-16, 7.451e-16])



In [17]:
# matplotlib.use('module://matplotlib-kitty')
import l2hmc.utils.plot_helpers as ph
from pathlib import Path
from l2hmc.common import get_timestamp

tstamp = get_timestamp()
outdir = Path(f"./outputs/pt4dsu3/2023-07-19/{tstamp}")
ph.plot_dataset(dataset, outdir=outdir, title='Training')

No event loop hook running.
Using matplotlib backend: <object object at 0x1073d22d0>
[07/19/23 22:09:42][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/energy_ridgeplot.svg
[07/19/23 22:09:43][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/logprob_ridgeplot.svg
[07/19/23 22:09:44][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/logdet_ridgeplot.svg
[07/19/23 22:09:45][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/sldf_ridgeplot.svg
[07/19/23 22:09:46][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/sldb_ridgeplot.svg
[07/19/23 22:09:47][INFO][plot_helpers.py:1006] - Saving figure to: outputs/pt4dsu3/2023-07-19/2023-07-19-220942/ridgeplots/svgs/sld_ridgeplot.svg
[07/19/23 22:09:49][I

<xarray.Dataset>
Dimensions:    (chain: 9, leapfrog: 9, draw: 19)
Coordinates:
  * chain      (chain) int64 0 1 2 3 4 5 6 7 8
  * leapfrog   (leapfrog) int64 0 1 2 3 4 5 6 7 8
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
Data variables: (12/18)
    energy     (chain, leapfrog, draw) float64 -4.774e+03 -4.831e+03 ... nan nan
    logprob    (chain, leapfrog, draw) float64 -4.774e+03 -4.831e+03 ... nan nan
    logdet     (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... nan nan nan
    sldf       (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... nan nan nan
    sldb       (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... nan nan nan
    sld        (chain, leapfrog, draw) float64 0.0 0.0 0.0 0.0 ... nan nan nan
    ...         ...
    loss       (draw) float64 -0.7719 -0.7836 -0.7867 -0.8042 ... 0.0 0.0 0.0
    plaqs      (chain, draw) float64 0.2526 0.2629 0.2779 0.2908 ... nan nan nan
    sinQ       (chain, draw) float64 0.002607 -0.0002384 0.000401 ..

In [ ]:
import l2hmc.utils.plot_helpers as ph
from l2hmc.common import di

In [ ]:
# fig, ax = plt.subplots()
figax = plot_metric(
    history['plaqs'],
    name='Plaqs (Training)',
    marker='.'
)
fig, ax = figax
fig.savefig('4dSU3-train-plaqs-2023-07-19.svg', bbox_inches='tight')

In [18]:
state = ptExpSU3.trainer.dynamics.random_state(6.0)
acc = torch.ones(state.x.shape[0])
m, mb = ptExpSU3.trainer.dynamics._get_mask(0)
loss = torch.tensor([0.])
with torch.autograd.detect_anomaly(check_nan=True):  # flake8: noqa  pyright:ignore
    ptExpSU3.trainer.optimizer.zero_grad()
    # state_vb, logdet_vf = ptExpSU3.trainer.dynamics._update_v_bwd(
    #     step=0, state=state_vf
    # )
    # console.log(f'TRAIN STEP: {step}')
    x, metrics = ptExpSU3.trainer.train_step((state.x, state.beta))
    loss = metrics['loss']
    # loss_xb = ptExpSU3.trainer.calc_loss(state_xb.x, state.x, acc=acc)
    # loss_xf = ptExpSU3.trainer.calc_loss(state_xf.x, state.x, acc=acc)
    # loss_ = loss_xb + loss_xf
    # loss = ptExpSU3.trainer.backward_step(loss_)
    console.print(f"loss: {loss:.5f}")

loss: -0.04936



In [19]:
state = ptExpSU3.trainer.dynamics.random_state(6.0)
acc = torch.ones(state.x.shape[0])
m, mb = ptExpSU3.trainer.dynamics._get_mask(0)
loss = torch.tensor([0.])
step = 0
from l2hmc.configs import State
with torch.autograd.detect_anomaly(check_nan=True):  # flake8: noqa  pyright:ignore
    ptExpSU3.trainer.optimizer.zero_grad()
    # sumlogdet = torch.zeros(state.x.shape[0], device=self.device)
    state_vf1, logdet = ptExpSU3.trainer.dynamics._update_v_fwd(step, state)
    sumlogdet = logdet
    # state_ = State(state.x, vf1, state.beta)
    state_xf1, logdet = ptExpSU3.trainer.dynamics._update_x_fwd(step, state_vf1, m, first=True)
    sumlogdet = sumlogdet + logdet
    state_xf2, logdet = ptExpSU3.trainer.dynamics._update_x_fwd(step, state_xf1, mb, first=False)
    sumlogdet = sumlogdet + logdet
    state_vf2, logdet = ptExpSU3.trainer.dynamics._update_v_fwd(step, state_xf2)
    sumlogdet = sumlogdet + logdet
    # state_, logdet = ptExpSU3.trainer.dynamics._forward_lf(step=0, state=state)
    # state_, logdet = ptExpSU3.trainer.dynamics._update_x_fwd(
    #     step=0, state=state, m=m, first=True
    # )
    # state_xb, logdet_xb = ptExpSU3.trainer.dynamics._update_x_bwd(
    #     step=0, state=state_xf, m=m, first=True
    # )
    # state_vb, logdet_vf = ptExpSU3.trainer.dynamics._update_v_bwd(
    #     step=0, state=state_vf
    # )
    # loss_xb = ptExpSU3.trainer.calc_loss(state_xb.x, state.x, acc=acc)
    loss_ = ptExpSU3.trainer.calc_loss(state_xf1.x, state.x, acc=acc)
    # loss_ = loss_xb + loss_xf
    loss = ptExpSU3.trainer.backward_step(loss_)
    console.print(f"loss: {loss.item():.5f}")

loss: -0.00020



In [20]:
state = ptExpSU3.trainer.dynamics.random_state(6.0)
acc = torch.ones(state.x.shape[0])
m, mb = ptExpSU3.trainer.dynamics._get_mask(0)
loss = torch.tensor([0.])
with torch.autograd.detect_anomaly(check_nan=True):  # flake8: noqa  pyright:ignore
    ptExpSU3.trainer.optimizer.zero_grad()
    state_vf, logdet_vf = ptExpSU3.trainer.dynamics._update_v_fwd(
        step=0, state=state
    )
    state_vb, logdet_vf = ptExpSU3.trainer.dynamics._update_v_bwd(
        step=0, state=state_vf
    )
    loss_ = ptExpSU3.trainer.calc_loss(state.x, state_vf.x, acc=acc)
    loss = ptExpSU3.trainer.backward_step(loss_)
    console.print(f"loss: {loss.item():.5f}")

loss: 0.00000



In [21]:
from torch import autograd
state = ptExpSU3.trainer.dynamics.random_state(6.0)
acc = torch.ones(state.x.shape[0])
m, mb = ptExpSU3.trainer.dynamics._get_mask(0)
loss = torch.tensor([0.])
# ptExpSU3.trainer.dynamics.init_weights(
#     method='uniform',
#     min=-1e-32,
#     max=1e-32,
#     bias=True,
#     xeps=0.001,
#     veps=0.001,
# )

with autograd.detect_anomaly(check_nan=True):
    ptExpSU3.trainer.optimizer.zero_grad()
    state_vf, logdet_vf = ptExpSU3.trainer.dynamics._update_v_fwd(
        step=0, state=state
    )
    state_xf, logdet_xf = ptExpSU3.trainer.dynamics._update_x_fwd(
        step=0, state=state_vf, m=m, first=True
    )
    console.print(f"state_xf.x.shape: {state_xf.x.shape}")
    avg, diff = ptExpSU3.trainer.g.checkSU(state_xf.x)
    console.print(f"avg: {avg}, diff: {diff}")
    loss_ = ptExpSU3.trainer.calc_loss(state.x, state_xf.x, acc=acc)
    loss = ptExpSU3.trainer.backward_step(loss_)
    console.print(f"loss: {loss.item():.5f}")

state_xf.x.shape: torch.Size([4, 4, 4, 4, 4, 8, 3, 3])

avg: tensor[4] f64 x∈[8.420e-08, 1.064e-07] μ=9.412e-08 σ=1.068e-08 grad SqrtBackward0 [8.420e-08, 1.064e-07, 8.622e-08, 9.967e-08], diff: tensor[4] f64 x∈[1.942e-07, 2.979e-07] μ=2.382e-07 σ=4.871e-08 grad SqrtBackward0 [1.942e-07, 2.979e-07, 2.030e-07, 2.577e-07]

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Cell In[19], line 27
     25 console.print(f"avg: {avg}, diff: {diff}")
     26 loss_ = ptExpSU3.trainer.calc_loss(state.x, state_xf.x, acc=acc)
---> 27 loss = ptExpSU3.trainer.backward_step(loss_)
     28 console.print(f"loss: {loss.item():.5f}")

File ~/projects/saforem2/l2hmc-qcd/src/l2hmc/trainers/pytorch/trainer.py:1283, in Trainer.backward_step(self, loss)
   1281 else:
   1282     if self.grad_scaler is not None:
-> 1283         self.grad_scaler.scale(loss).backward()  # type:ignore
   1284         if self.config

In [22]:
from torch import autograd
from l2hmc.dynamics.pytorch.dynamics import sigmoid

step = 0
dynamics = ptExpSU3.trainer.dynamics
state = ptExpSU3.trainer.dynamics.random_state(6.0)
acc = torch.ones(state.x.shape[0])
m, mb = ptExpSU3.trainer.dynamics._get_mask(0)
loss = torch.tensor([0.])
ptExpSU3.trainer.dynamics.init_weights(
    method='uniform',
    min=-1e-32,
    max=1e-32,
    bias=True,
    xeps=0.001,
    veps=0.001,
)

with autograd.detect_anomaly(check_nan=True):
    eps = sigmoid(dynamics.veps[step].log())
    force = dynamics.grad_potential(state.x, state.beta)
    s, t, q = dynamics._call_vnet(step, (state.x, force))
    logjac = eps * s / 2.  # jacobian factor, also used in exp_s below
    logdet = dynamics.flatten(logjac).sum(1)
    force = force.reshape_as(state.v)
    exp_s = (logjac.exp()).reshape_as(state.v)
    exp_q = (eps * q).exp().reshape_as(state.v)
    t = t.reshape_as(state.v)
    vf = (exp_s * state.v) - (0.5 * eps * (force * exp_q + t))
    if dynamics.config.group == 'SU3':
        vf = dynamics.g.projectTAH(vf)
    loss_ = ptExpSU3.trainer.calc_loss(state.v, vf, acc=acc)
    loss_.register_hook(lambda grad: grad.clamp_(max=1.0))
    loss_.register_hook(lambda grad: console.print(f"grad: {grad}"))
    ptExpSU3.trainer.optimizer.zero_grad()
    loss_.backward()
    torch.nn.utils.clip_grad.clip_grad_norm(dynamics.parameters(), max_norm=1.0)
    ptExpSU3.trainer.optimizer.step()
    # loss = ptExpSU3.trainer.backward_step(loss_)
    console.print(f"loss: {loss_.item():.5f}")

grad: 1.0

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Cell In[80], line 36
     34 loss_.register_hook(lambda grad: console.print(f"grad: {grad}"))
     35 ptExpSU3.trainer.optimizer.zero_grad()
---> 36 loss_.backward()
     37 torch.nn.utils.clip_grad.clip_grad_norm(dynamics.parameters(), max_norm=1.0)
     38 ptExpSU3.trainer.optimizer.step()

File ~/miniconda3/envs/py310/lib/python3.10/site-packages/torch/_tensor.py:491, in Tensor.backward(self, gradient, retain_graph, create_graph, inputs)
    481 if has_torch_function_unary(self):
    482     return handle_torch_function(
    483         Tensor.backward,
    484         (self,),
   (...)
    489         inputs=inputs,
    490     )
--> 491 torch.autograd.backward(
    492     self, gradient, retain_graph, create_graph, inputs=inputs
    493 )

File ~/miniconda3/envs/py310/lib/python3.10/site-packages/torch/autograd/__init__

In [23]:
history = {}
x = state.x
for step in range(50):
    console.log(f'TRAIN STEP: {step}')
    x, metrics = ptExpSU3.trainer.train_step((x, state.beta))
    if (step > 0 and step % 2 == 0):
        print_dict(metrics, grab=True)
    if (step > 0 and step % 1 == 0):
        for key, val in metrics.items():
            try:
                history[key].append(val)
            except KeyError:
                history[key] = [val]

x = ptExpSU3.trainer.dynamics.unflatten(x)
console.log(f"checkSU(x_train): {g.checkSU(x)}")
plot_metrics(history, title='train', marker='.')

[11:09:36] TRAIN STEP: 0

[W reducer.cpp:1353] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())


[11:09:38] TRAIN STEP: 1

[11:09:39] TRAIN STEP: 2

[07/17/23 11:09:40][INFO][common.py:97] - energy: torch.Size([9, 4]) torch.float64 
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
logprob: torch.Size([9, 4]) torch.float64 
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
logdet: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
sldf: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
sldb: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. 

...

[11:09:44] TRAIN STEP: 7

[11:09:45] TRAIN STEP: 8

[07/17/23 11:09:45][INFO][common.py:97] - energy: torch.Size([9, 4]) torch.float64 
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
logprob: torch.Size([9, 4]) torch.float64 
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
logdet: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
sldf: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
sldb: torch.Size([9, 4]) torch.float64 
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0. 

           TRAIN STEP: 9

KeyboardInterrupt: 